In [1]:
import numpy as np
import argparse
import time
import cv2
import os

In [2]:
image_path = "images/lot2.jpg" # image file
yolo_path = "../yolo"  # yolo directory
confidence_min = 0.2
nms_thresh = 0.3

# load the COCO class labels
labels_path = os.path.sep.join([yolo_path, "coco.names"])
LABELS = open(labels_path).read().strip().split("\n")
# initialize color list
np.random.seed(42)
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3),dtype="uint8")

In [3]:
# paths to weights and model configuration
weights_path = os.path.sep.join([yolo_path, "yolov3.weights"])
config_path = os.path.sep.join([yolo_path, "yolov3.cfg"])
print("[INFO] loading YOLO from disk...")
net = cv2.dnn.readNetFromDarknet(config_path, weights_path)


[INFO] loading YOLO from disk...


In [4]:
# load input image
image = cv2.imread(image_path)
(H, W) = image.shape[:2]
# get output layer names
ln = net.getLayerNames()
try:
    ln = [ln[i - 1] for i in net.getUnconnectedOutLayers()]
except:
    ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

# construct a blob from the input image and then perform a forward
# pass for bboxes
blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416), swapRB=True, crop=False)
net.setInput(blob)
start = time.time()
layerOutputs = net.forward(ln)
end = time.time()
# timing verbose
print("[INFO] YOLO took {:.6f} seconds".format(end - start))


[INFO] YOLO took 0.198235 seconds


In [5]:
bboxes = []
confidences = []
classIDs = []

# loop over layer outputs
for output in layerOutputs:
    # loop over detections
    for detection in output:
        scores = detection[5:]
        classID = np.argmax(scores)
        confidence = scores[classID]
        # filter out bad predictions using confidence_min
        if confidence > confidence_min:
            # scale the bounding box coordinates back relative to the
            # size of the image, keeping in mind that YOLO actually
            # returns the center (x, y)-coordinates of the bounding
            # box followed by the boxes' width and height
            box = detection[0:4] * np.array([W, H, W, H])
            (centerX, centerY, width, height) = box.astype("int")
            # use the center (x, y)-coordinates to derive the top and
            # and left corner of the bounding box
            x = int(centerX - (width / 2))
            y = int(centerY - (height / 2))
            # update our list of bounding box coordinates, confidences,
            # and class IDs
            bboxes.append([x, y, int(width), int(height)])
            confidences.append(float(confidence))
            classIDs.append(classID)

In [6]:
# apply non-maxima suppression using cv2
final_bbox_ids = cv2.dnn.NMSBoxes(bboxes, confidences, confidence_min, nms_thresh)

In [7]:
# plot
if len(final_bbox_ids) > 0:
    # loop over indexes
    for i in final_bbox_ids.flatten():
        # extract the bounding box values from boxes
        (x, y) = (bboxes[i][0], bboxes[i][1])
        (w, h) = (bboxes[i][2], bboxes[i][3])
        # draw a bounding box rectangle and label on the image
        color = [int(c) for c in COLORS[classIDs[i]]]
        cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
        text = "{}: {:.4f}".format(LABELS[classIDs[i]], confidences[i])
        cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
# show the output image
cv2.imshow("Image", image)
cv2.waitKey(0)
